In [1]:
import pandas as pd
import requests
import edgar_functions as edgar
headers = {"User-Agent": "jacobg0121@gmail.com"} 
import os

In [2]:
def get_facts(ticker, headers=headers):
    cik = edgar.cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts

In [3]:
facts = get_facts("AAPL")
facts["facts"]["us-gaap"]["RevenueFromContractWithCustomerExcludingAssessedTax"]['units']['USD']

[{'start': '2016-09-25',
  'end': '2017-09-30',
  'val': 229234000000,
  'accn': '0000320193-19-000119',
  'fy': 2019,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2019-10-31',
  'frame': 'CY2017'},
 {'start': '2017-10-01',
  'end': '2017-12-30',
  'val': 88293000000,
  'accn': '0000320193-19-000010',
  'fy': 2019,
  'fp': 'Q1',
  'form': '10-Q',
  'filed': '2019-01-30'},
 {'start': '2017-10-01',
  'end': '2017-12-30',
  'val': 88293000000,
  'accn': '0000320193-19-000119',
  'fy': 2019,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2019-10-31',
  'frame': 'CY2017Q4'},
 {'start': '2017-10-01',
  'end': '2018-03-31',
  'val': 149430000000,
  'accn': '0000320193-19-000066',
  'fy': 2019,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2019-05-01'},
 {'start': '2017-12-31',
  'end': '2018-03-31',
  'val': 61137000000,
  'accn': '0000320193-19-000066',
  'fy': 2019,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2019-05-01'},
 {'start': '2017-12-31',
  'end': '2018-03-31',
  'val': 61137000000,
  'a

In [4]:
def facts_DF(ticker, headers=headers):
    facts = get_facts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [5]:
facts, label_dict = facts_DF("aapl")

In [6]:
def annual_facts(ticker, headers=headers):
    accession_nums = edgar.get_filtered_filings(
        ticker, ten_k=True, just_accession_numbers=True
    )
    df, label_dict = facts_DF(ticker, headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [7]:
def quarterly_facts(ticker, headers=headers):
    accession_nums = get_filtered_filings(
        ticker, ten_k=False, just_accession_numbers=True
    )
    df, label_dict = facts_DF(ticker, headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=["fact", "end"], keep="last")
    pivot = ten_q.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [8]:
quarterly = quarterly_facts('aapl')

NameError: name 'get_filtered_filings' is not defined

In [ ]:
def save_dataframe_to_csv(dataframe, folder_name, ticker, statement_name, frequency):
    directory_path = os.path.join(folder_name, ticker)
    os.makedirs(directory_path, exist_ok=True)
    file_path = os.path.join(directory_path, f"{statement_name}_{frequency}.csv")
    dataframe.to_csv(file_path)
    return None